In [78]:
import os
import sqlite3
import pandas as pd
import numpy as np

import openpyxl as op
from openpyxl import load_workbook

from datetime import date, datetime

In [34]:
# CONSTANTS
FILE_EXTENSION = ".txt"
FILEPATH = "./files"
TABLES_SEPARATER = '\n\n'
TABLE_DATA_SEPRATER = '\n'
TABLENAME_STARTING_DELIMITER = "*** "
TABLENAME_ENDING_DELIMTER = " ***"
COLUMNS_SEPARATER = '\t'
TABLE_DATA_SEPARATER = '\t'
RESULT_FILENAME = "Implementation_check_" + str(datetime.now().strftime("%Y-%m-%d_%H-%M-%S")) + ".xlsx"
SHEET_NAME_EXTRACTOR = ""

# Replaces any spaces, &, - characters into underscore
TABLENAME_CLEANING_CHARACTERS = [(" ", "_"),("&", "_"),("-", "_")]

In [29]:
requiredFilesList = list()

with os.scandir(FILEPATH) as currentDirectory:
    for entry in currentDirectory:
        
        # Criteria for selecting the required files
        criteriaSet = set()
        criteriaSet.add(entry.name.endswith(FILE_EXTENSION))
        criteriaSet.add((entry.name.lower().find("before") > -1) or (entry.name.lower().find("after") > -1))
        if False not in criteriaSet:
            requiredFilesList.append(entry.name)

# Updating the file names to include the path
requiredFilesList = [FILEPATH + "/" + file for file in requiredFilesList]
requiredFilesList.sort()
requiredFilesList

['./files/test2After.txt', './files/test2Before.txt']

In [93]:
resultList = dict()
index = 1

for entry in requiredFilesList:

    fileType = "after"
    if entry[:-len(FILE_EXTENSION)][-1] == "e": fileType = "before"
    
    comparisonList = list()
    dbName = entry[ : -len(FILE_EXTENSION) - len(fileType)]
    con = sqlite3.connect(dbName + '.db')
    cur = con.cursor()
    file=open(entry,"r")
    
    for table in file.read().split(TABLES_SEPARATER):
        
        # Get the data of individual tables
        tableInfo = table.split(TABLE_DATA_SEPRATER)
        
        if tableInfo != '' and len(tableInfo) > 1:
            
            # Cleaning and storing the table name
            tableName = tableInfo[0][len(TABLENAME_STARTING_DELIMITER) : -len(TABLENAME_ENDING_DELIMTER)]
            tableName = tableName + "_" + fileType
            for clean_char in TABLENAME_CLEANING_CHARACTERS:
                tableName = tableName.replace(clean_char[0], clean_char[1])
            
            # Cleaning and storing the table columns
            tableColumns = list()
            for column in tableInfo[1].split(COLUMNS_SEPARATER):
                tableColumns.append(column.replace(".", "_"))
            tableColumns = tuple(tableColumns)
            
            # Creating tables and execution
            tableCreationQuery = 'CREATE TABLE {table_name} {table_columns}'
            tableCreationQuery = tableCreationQuery.format(table_name = tableName, table_columns = tableColumns)
            try:
                cur.execute(tableCreationQuery)
            except sqlite3.OperationalError:
                pass
            
            for data in tableInfo[2:]:
                
                # Creating table data and execution
                row_data = tuple(data.split(TABLE_DATA_SEPARATER))
                tableDataInsertionQuery = 'INSERT INTO {table_name} VALUES {table_data}'
                tableDataInsertionQuery = tableDataInsertionQuery.format(table_name = tableName, table_data = row_data)
                cur.execute(tableDataInsertionQuery)
            
            con.commit()
            
            if fileType == "before":
                comparisonList.append((tableName, tableColumns))

    if fileType == "before":
        
        resultList[dbName] = list()
        resultFileSheetName = dbName[len(FILEPATH + "/" + SHEET_NAME_EXTRACTOR): ]
        if not os.path.isfile(RESULT_FILENAME):
            wb = op.Workbook()
            wb.save(RESULT_FILENAME)
        wb = load_workbook(filename = RESULT_FILENAME)
        ws = wb.create_sheet(title = resultFileSheetName)
        wb.save(RESULT_FILENAME)
        
        for tableComparison in comparisonList:

            beforeTable = tableComparison[0]
            afterTable = tableComparison[0][:-len("before")] + "after"
            columns = tableComparison[1]
            
            tableResult = dict()
            tableResult["Table Name"] = beforeTable[:-len("_before")]
            tableResult["Columns"] = columns
            
            displayColumns1 = "b." + columns[0] + " AS before_date, a." + columns[0] + " AS after_date, "
            displayColumns2 = "a." + columns[0] + " AS before_date, b." + columns[0] + " AS after_Date, "
            for column in columns[1:-1]:
                displayColumns1 += "a." + column + ", "
                displayColumns2 += "a." + column + ", "
            displayColumns2 += "a." + columns[-1] + " AS count_before, b." + columns[-1] + " AS count_after"
            displayColumns1 += "b." + columns[-1] + " AS count_before, a." + columns[-1] + " AS count_after"
            
            joinCondition = ""
            for column in columns[1:-2]:
                joinCondition += "a." + column + " = b." + column + " AND "
            joinCondition += "a." + columns[-2] + " = b." + columns[-2]
            
            query1 = "SELECT {display_columns} from {after_table} AS a LEFT JOIN {before_table} AS b ON {join_condition}"
            query1 = query1.format(display_columns = displayColumns1, after_table = afterTable, before_table = beforeTable, join_condition = joinCondition)
            query2 = "SELECT {display_columns} from {before_table} AS a LEFT JOIN {after_table} AS b ON {join_condition}"
            query2 = query2.format(display_columns = displayColumns2, before_table = beforeTable, after_table = afterTable, join_condition = joinCondition)
            query = "{} UNION {}".format(query1, query2)
            
            new_df = pd.read_sql(query, con)
            new_df[["count_before","count_after"]] = new_df[["count_before","count_after"]].fillna(0)
            new_df[["count_before","count_after"]] = new_df[["count_before","count_after"]].applymap(int)
            new_df["Difference (after - before)"] = new_df["count_after"] - new_df["count_before"]
            new_df = new_df.sort_values(new_df.columns[2:-3].tolist())
            
    con.close()
    file.close()
    
    if fileType == "before":
        os.remove(dbName + '.db')
     

In [99]:
new_df

,before_date,after_date,Column_2,Column_3,Column_4,count_before,count_after,Difference (after - before)
0,None,02-11-99,BC,02,001,0,5,5
1,02-11-99,None,BC,01,002,5,0,-5
2,02-11-99,02-11-99,AD,01,001,1,1,0
3,02-11-99,02-11-99,AD,01,002,2,2,0
4,02-11-99,02-11-99,AD,02,001,3,3,0
5,02-11-99,02-11-99,BC,01,001,4,4,0
